In [12]:
import os
import numpy as np
import random
import cv2
from tensorflow.keras.utils import Sequence
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from tqdm import tqdm  # For progress bars
from sklearn.model_selection import train_test_split
import tensorflow as tf

# --- Load Dataset ---


In [13]:
# Define your dataset paths (adjust the paths as needed)
dataset_dirs = [
    '/kaggle/input/youtube-faces-with-facial-keypoints/youtube_faces_with_keypoints_full_1/youtube_faces_with_keypoints_full_1',
    '/kaggle/input/youtube-faces-with-facial-keypoints/youtube_faces_with_keypoints_full_2/youtube_faces_with_keypoints_full_2',
    '/kaggle/input/youtube-faces-with-facial-keypoints/youtube_faces_with_keypoints_full_3/youtube_faces_with_keypoints_full_3',
    '/kaggle/input/youtube-faces-with-facial-keypoints/youtube_faces_with_keypoints_full_4/youtube_faces_with_keypoints_full_4'
]

# Collecting all files from the specified directories
files = []
for dataset_dir in dataset_dirs:
    files += [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# Debugging step: print the number of files found
print(f"Number of files found: {len(files)}")
if len(files) == 0:
    print("No files found! Please check the directory paths and file structure.")
else:
    # Perform train-test split if files are found
    train_files, val_files = train_test_split(files, test_size=0.2, random_state=42)

    print(f"Training set size: {len(train_files)} files")
    print(f"Validation set size: {len(val_files)} files")

Number of files found: 2194
Training set size: 1755 files
Validation set size: 439 files


# --- Data Generator Class ---


In [14]:
class DataGenerator(Sequence):
    def __init__(self, files, batch_size=32, sample_ratio=0.1, img_size=(256, 256), shuffle=True, return_names=False):
        self.files = files
        self.batch_size = batch_size
        self.sample_ratio = sample_ratio
        self.img_size = img_size
        self.shuffle = shuffle
        self.return_names = return_names
        
        if self.shuffle:
            random.shuffle(self.files)

    def __len__(self):
        return int(np.floor(len(self.files) / self.batch_size))

    def __getitem__(self, index):
        batch_files = self.files[index * self.batch_size:(index + 1) * self.batch_size]

        images, bboxes, landmarks_2d, landmarks_3d, image_names = [], [], [], [], []
        for npz_file in batch_files:
            data = np.load(npz_file)
            color_images = data['colorImages']
            bboxes_data = data['boundingBox']
            landmarks2D_data = data['landmarks2D']
            landmarks3D_data = data['landmarks3D']

            num_frames = color_images.shape[-1]
            sampled_indices = random.sample(range(num_frames), int(self.sample_ratio * num_frames))

            filename = os.path.basename(npz_file).split('.')[0]

            for idx in sampled_indices:
                img = color_images[..., idx]
                img = cv2.resize(img, self.img_size)
                img = img / 255.0  # Normalize

                images.append(img)  # Use original images

                bboxes.append(bboxes_data[..., idx])
                landmarks_2d.append(landmarks2D_data[..., idx])
                landmarks_3d.append(landmarks3D_data[..., idx])
                image_names.append(filename)

        images = np.array(images)

        if self.return_names:
            return images, images, image_names  # Return image names for visualization
        else:
            return images, images  # Don't return names during training

    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.files)

In [15]:
# --- Train-Validation Split ---
train_files, val_files = train_test_split(files, test_size=0.2, random_state=42)

# --- Data Generators ---
train_generator = DataGenerator(files=train_files, batch_size=8, sample_ratio=0.05, img_size=(256, 256), shuffle=True)
val_generator = DataGenerator(files=val_files, batch_size=8, sample_ratio=0.05, img_size=(256, 256), shuffle=False)

# --- Autoencoder Model ---


In [16]:
def build_autoencoder():
    input_img = keras.Input(shape=(256, 256, 3))  # Keep this as it is, can reduce later if necessary

    # Encoder
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoded = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # Autoencoder model
    autoencoder = keras.Model(input_img, decoded)
    autoencoder.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='mse')
    autoencoder.summary()

    return autoencoder

# --- Multi-GPU Strategy ---


In [17]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    autoencoder = build_autoencoder()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 256, 256, 128)  │         3,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 256, 256, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 64)   │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 256, 256, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 256, 256, 3)    │         3,459 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 201,795 (788.26 KB)

 Trainable params: 201,347 (786.51 KB)

 Non-trainable params: 448 (1.75 KB)

# --- Model Training with Callbacks ---


In [ ]:
# --- Callbacks ---
callbacks = [
    ModelCheckpoint('/kaggle/working/best_model.keras', save_best_only=True, monitor='val_loss', mode='min'),
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
]

# --- Training Loop ---
autoencoder.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=callbacks
)

Epoch 1/50


2024-09-26 03:56:01.249385: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/cond/else/_186/cond/StatefulPartitionedCall/functional_1_1/dropout_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


219/219 ━━━━━━━━━━━━━━━━━━━━ 534s 2s/step - loss: 0.0258 - val_loss: 0.0685 - learning_rate: 1.0000e-04
Epoch 2/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 304s 1s/step - loss: 0.0119 - val_loss: 0.0391 - learning_rate: 1.0000e-04
Epoch 3/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 283s 1s/step - loss: 0.0093 - val_loss: 0.0143 - learning_rate: 1.0000e-04
Epoch 4/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 291s 1s/step - loss: 0.0078 - val_loss: 0.0067 - learning_rate: 1.0000e-04
Epoch 5/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 286s 1s/step - loss: 0.0070 - val_loss: 0.0085 - learning_rate: 1.0000e-04
Epoch 6/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 286s 1s/step - loss: 0.0058 - val_loss: 0.0076 - learning_rate: 1.0000e-04
Epoch 7/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 285s 1s/step - loss: 0.0054 - val_loss: 0.0068 - learning_rate: 1.0000e-04
Epoch 8/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 285s 1s/step - loss: 0.0051 - val_loss: 0.0067 - learning_rate: 1.0000e-04
Epoch 9/50
219/219 ━━━━━━━━━━━━━━━━━━━━ 284s 1s/step - loss: 0.0048 - val_loss: 0.0073 - le

# --- Save the model after training ---


In [ ]:
autoencoder.save("/kaggle/working/final_model.keras")

# --- Testing and Visualization ---


In [ ]:
test_generator = DataGenerator(files=val_files, batch_size=16, sample_ratio=0.05, img_size=(256, 256), shuffle=False, return_names=True)
test_images, _, test_names = test_generator[0]

reconstructed_images = autoencoder.predict(test_images[:5])

# --- Display Function ---

def display_comparison(original, reconstructed, names, n=5):
    plt.figure(figsize=(15, 5))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(original[i])
        plt.title(f"Original: {names[i]}")
        plt.axis("off")

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed[i])
        plt.title("Reconstructed")
        plt.axis("off")
    plt.show()

# --- Display Comparison ---
display_comparison(test_images[:5], reconstructed_images, test_names[:5], n=5)

# --- Save Comparisons ---


In [ ]:
def save_comparisons(original, enhanced, names, folder="/kaggle/working/prediction"):
    os.makedirs(folder, exist_ok=True)
    for i in range(len(original)):
        original_img = (original[i] * 255).astype(np.uint8)
        enhanced_img = (enhanced[i] * 255).astype(np.uint8)
        combined = np.hstack((original_img, enhanced_img))
        cv2.imwrite(os.path.join(folder, f"{names[i]}_comparison.png"), combined)

save_comparisons(test_images[:5], reconstructed_images, test_names[:5])
